In [312]:
import numpy as np
import pandas as pd
import pylab as pl
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from scipy.stats import threshold

Primeiramente importamos o dataset utilizando pandas

In [313]:
texto = pd.read_csv("../data/anger-ratings-0to1.train.txt", sep="\t", header=None)

In [314]:
texto.shape

(857, 4)

In [315]:
texto.sample(5)

,0,1,2,3
550,10550,@xandraaa5 @amayaallyn6 shut up hashtags are c...,anger,0.438
711,10711,Sometimes I think the British political landsc...,anger,0.354
776,10776,@ImpugnValkyrie *I frown and cup your cheeks i...,anger,0.304
148,10148,@cineworld 'Congratulations your Free 1 month ...,anger,0.667
622,10622,@SarahMi40605180 @kaileighhxo sorry to burst t...,anger,0.396


In [316]:
target = texto[3].values
data = texto[1].values

Valores de 'anger' maiores que 0.5 são elevados para 1, e menores para 0.

In [317]:
target = [1 if x >= 0.5 else 0 for x in target]

Decidimos utilizar 85% dos valores para treinamento e o resto para teste

In [318]:
slicePoint = int(len(data)*0.85)
slicePoint

728

Faremos o experimento de classificação de maneira ingênua utilizando o classificador k-nearest neighbors

In [319]:
knn = KNeighborsClassifier()

A tokenização do array de dados (data) é feita pelo algoritmo CountVectorizer, que armazena os dados em uma matriz esparsa e condensa os dados utilizando um vetor de vocabulário e apenas referenciando as frases a esse vetor dicionário.

In [320]:
cv = CountVectorizer()

In [321]:
trans = cv.fit_transform(data, target)

In [322]:
trans.shape

(857, 3436)

Os valores são inseridos no classificador

In [323]:
knn.fit(trans[:slicePoint], target[:slicePoint])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

Utilizando os valores de 6000 até o final do dataset para teste do preditor

In [324]:
testTarget = knn.predict(trans[slicePoint:])

In [325]:
results = np.in1d(testTarget, target[slicePoint:]).tolist()

O preditor funciona muito mal

In [326]:
float(results.count(True)) / float(len(results))

0.21705426356589147

Tentaremos uma nova abordagem, então, utilizando o algoritmo "term frequency–inverse document frequency", a fim de priorizar palavras importantes na classificação do texto.

In [327]:
tfidf = TfidfTransformer()

In [328]:
tfidfout = tfidf.fit_transform(trans)

In [329]:
tfidfout.shape

(857, 3436)

In [330]:
knn = KNeighborsClassifier()

In [331]:
knn.fit(tfidfout[:slicePoint], target[:slicePoint])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [332]:
testTarget2 = knn.predict(tfidfout[slicePoint:])

In [333]:
results2 = np.in1d(testTarget2, target[slicePoint:]).tolist()

Houve uma melhora de pouco mais de 20% na taxa de acerto do modelo utilizando o TFIDF

In [334]:
float(results2.count(True)) / float(len(results2))

0.4496124031007752

Porém o problema da classificação ainda é a utilização do classificador k-nearest neighbors, que não é apropriado para nosso dataset. Tentaremos, então, utilizando o algoritmo de Logistic Regression.

In [335]:
lr = LogisticRegression()

In [336]:
lr.fit(trans[:slicePoint], target[:slicePoint])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [337]:
testTarget3 = lr.predict(trans[slicePoint:])

In [338]:
results3 = np.in1d(testTarget3, target[slicePoint:]).tolist()

Obtemos, agora, uma taxa de acerto de 31% para o nosso dataset. Um ganho de 20% em relação ao algoritmo anterior.

In [339]:
float(results3.count(True)) / float(len(results3))

0.6666666666666666

Utilizaremos, então, o algoritmo SGDClassifier

In [370]:
sgd = SGDClassifier(loss='hinge', penalty='l1', alpha=1e-3, random_state=42)

In [371]:
sgd.fit(trans[:slicePoint], target[:slicePoint])

SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=5, n_iter=None,
       n_jobs=1, penalty='l1', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [372]:
testTarget4 = sgd.predict(trans[slicePoint:])

In [373]:
results4 = np.in1d(testTarget4, target[slicePoint:]).tolist()

Utilizando esse algoritmo alcançamos uma taxa de acerto consideravelmente maior de 88%

In [374]:
float(results4.count(True)) / float(len(results4))

0.6744186046511628